In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install albumentations==0.4.6
!pip install pycocotools
!pip install torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 KB 14.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for albumentations: filename=albumentations-0.4.6-py3-none-any.whl size=65173 sha256=87652dd59eb21bcf3b9692da97965a1cfc8405c96334bb7564f47c349d1e0d1c
  Stored in directory: /root/.cache/pip/wheels/d2/e3/0b/99a239413035502833a7b07283894243fddf5ce3aa720ca8dd
Successfully built albumentations
  Attempting uninstall: albumentations
    Found existing installation: albumentations 1.2.1
    Uninstalling albumentations-1.2.1:
      Successfully uninstalled albumentations-1.2.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/Skola/py-AI/wildfire/test_obj')

In [4]:
%%shell

git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.8.2

cp references/detection/utils.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/engine.py ../
cp references/detection/coco_utils.py ../

Cloning into 'vision'...
remote: Enumerating objects: 289636, done.
remote: Counting objects: 100% (10762/10762), done.
remote: Compressing objects: 100% (708/708), done.
remote: Total 289636 (delta 10110), reused 10627 (delta 10030), pack-reused 278874
Receiving objects: 100% (289636/289636), 586.92 MiB | 16.99 MiB/s, done.
Resolving deltas: 100% (265217/265217), done.
Note: switching to 'v0.8.2'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at 2f40a483d7 [v0.8.X] .circleci: Add Python 3.9 to CI (#3063)


In [5]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torchvision.transforms as transform
import albumentations as A
import torchvision
import torch
import cv2

from torchvision.models.detection.rpn import AnchorGenerator
from albumentations.pytorch.transforms import ToTensorV2
from torchvision.models.detection import FasterRCNN
from engine import train_one_epoch, evaluate
from torch.utils.data import Dataset, DataLoader
from typing import Any

In [6]:
test_df = pd.read_csv('/content/drive/MyDrive/Skola/py-AI/wildfire/test_obj/test_annotations.csv')
test_images = '/content/drive/MyDrive/Skola/py-AI/wildfire/test_obj/test'

test_df.head()

,filename,width,height,class,xmin,ymin,xmax,ymax
0,ck0t4z1yrkmfv0794t2oqhd3f_jpeg.rf.17653d46c3ee...,640,480,smoke,514,209,616,285
1,ck0ujmglz85u10a468qt0d4fc_jpeg.rf.18d02755e8c9...,640,480,smoke,162,210,635,302
2,ck0uivtpc841h0a46ydgo7566_jpeg.rf.00134176dc29...,640,480,smoke,308,206,392,252
3,ck0nehpd69bax0721onacbe33_jpeg.rf.02ed50fbcb97...,640,480,smoke,274,229,454,311
4,ck0ow7vs07tuz08485n4yz5si_jpeg.rf.179fc0e59422...,640,480,smoke,386,218,607,285


In [7]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

LABELS = test_df['class'].unique()
NUM_OF_CLASSES = len(LABELS)+1

SAVE_PATH = '/content/drive/MyDrive/Skola/py-AI/wildfire/test_obj/models/'
MODEL_NAME = 'model_new.pt'

SAMPLES = 16

In [8]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(
    pretrained=False,
    num_classes=NUM_OF_CLASSES,
    map_location=DEVICE
    )
IN_FEATURES = model.roi_heads.box_predictor.cls_score.in_features

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [9]:
def collate_fn(batch: tuple) -> tuple:
  return tuple(zip(*batch))

In [10]:
def plot_predict_samples(loader, model, device, threshold) -> None:
  _images, _targets =next(iter(loader))
  images = [img.to(device) for img in _images]
  targets = [{k: v for k,v in target.items()} for target in _targets]

  true_boxes = targets[0]['boxes'].cpu().numpy().astype(np.int32)
  sample = images[0].permute(1,2,0).cpu().numpy()

  model.to(device)
  model.eval()
  cpu_device = torch.device('cpu')

  _outputs = model(images)
  outputs = [{k: v.to(cpu_device) for k, v in target.items()} for target in _outputs]

  fig, ax = plt.subplots(1, 1, figsize=(16,8))

  for box in true_boxes:
    cv2.rectangle(
          img=sample,
          pt1=(box[0], box[1]),
          pt2=(box[2], box[3]),
          color=(1,0,0),
          thickness=1
      )
    
  pred_boxes = outputs[0]['boxes'].data.cpu().numpy().astype(np.int32)
  pred_scores = outputs[0]['scores'].data.cpu().numpy()
  pred_labels = outputs[0]['labels'].data.cpu().numpy().astype(np.int32)

  box_counter = 0
  for box, score, label in zip(pred_boxes, pred_scores, pred_labels):
    if score > threshold:
      box_counter += 1
      cv2.rectangle(
          img=sample,
          pt1=(box[0], box[1]),
          pt2=(box[2], box[3]),
          color=(0,0,1),
          thickness=1
      )

      cv2.putText(
          img=sample,
          text=f'{LABELS[label-1]} : {score: .2%}%',
          org=(box[0], box[1]-10),
          fontFace=cv2.FONT_HERSHEY_DUPLEX,
          fontScale=0.3,
          color=(0,0,0),
          thickness=1
      )
    print(f'Predicted {box_counter} (blue) : True {len(true_boxes)} (red)')
    ax.set_axis_off()
    ax.imshow((sample * 255).astype(np.uint8))
  

In [11]:
class LabelMap:
  def __init__(self, labels: list) -> None:
    self._map = {c: i+1 for i, c in enumerate(labels)}
    self.reversed_map = {i: c for i, c in enumerate(labels)}

  def fit(self, df: pd.DataFrame, col: str) -> pd.DataFrame:
    df[col] = df[col].map(self._map)
    return df


def encode_label(df: pd.DataFrame, col: str, map_dict: dict) -> pd.DataFrame:
  df[col] = df[col].map(map_dict)
  return df

In [12]:
class WildfireDataset(Dataset):
  def __init__(self, df: pd.DataFrame, img_path: str, labels: list, transforms: Any = None, **kwargs) -> None:
    super().__init__(**kwargs)
    self.df = df
    self.img_path = img_path
    self.labels = labels
    self.images = self.df['filename'].unique()
    self.transforms = transforms

  def __len__(self) -> int:
    return len(self.images)

  def __getitem__(self, i: int) -> tuple:
    img_file = os.path.join(self.img_path, self.images[i])

    img = cv2.imread(img_file)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32)
    img = img/255.0

    img_data = self.df.loc[self.df['filename'] == self.images[i]]

    xmins = img_data['xmin'].values
    ymins = img_data['ymin'].values
    xmaxs = img_data['xmax'].values
    ymaxs = img_data['ymax'].values

    boxes = torch.as_tensor(np.stack([xmins, ymins, xmaxs, ymaxs], axis=1), dtype=torch.float32)
    labels = torch.as_tensor(img_data['class'].values, dtype=torch.int64)
    _id = torch.tensor([i])

    areas = (boxes[:,3] - boxes[:,1]) * (boxes[:,2] - boxes[:,0])
    areas = torch.as_tensor(areas, dtype=torch.float32)

    iscrowd = torch.zeros((len(labels),), dtype=torch.int64)

    target = dict()
    target['boxes'] = boxes
    target['labels'] = labels
    target['image_id'] = _id
    target['area'] = areas
    target['iscrowd'] = iscrowd

    if self.transforms:
      transformed = self.transforms(image=img, bboxes=boxes, labels=labels)
      img = transformed['image']
      target['boxes'] = torch.as_tensor(transformed['bboxes'], dtype=torch.float32)
    
    return torch.as_tensor(img, dtype=torch.float32), target

  def get_h_w(self, image: str) -> tuple:
    """Get height and width of image"""
    img_data = self.df.loc[self.df['filename'] == image]
    return img_data['width'].values[0], img_data['height'].values[0]

In [13]:
model.load_state_dict(torch.load(SAVE_PATH + MODEL_NAME))

<All keys matched successfully>

In [14]:
label_map = LabelMap(LABELS)
test_df = encode_label(test_df, 'class', label_map._map)

In [15]:
test_transform = A.Compose(
    [ToTensorV2(p=1)],
    bbox_params=A.BboxParams(format='pascal voc', label_fields=['labels'])
)

test_dataset = WildfireDataset(test_df, test_images, LABELS, test_transform)
test_dataloader = DataLoader(
    test_dataset,
    batch_size=4,
    shuffle=True,
    num_workers=2,
    collate_fn=collate_fn
)

In [ ]:
for i in range(SAMPLES):
  plot_predict_samples(test_dataloader, model, DEVICE, 0.6)

In [ ]:
from PIL import Image

def get_relevant_scores(threshold: float, boxes: list, scores: list, labels: list) -> tuple:
    """
    Get score over a set lower accuracy limit
    :param labels: list; predicted labels
    :param scores: list; predicted accuracy scores
    :param boxes: list; predicted bounding boxes
    :return: tuple[list, list, list];
    """
    x = len([score for score in scores if score >= threshold])
    return boxes[:x], scores[:x], labels[:x]

In [ ]:
from torchmetrics.detection.mean_ap import MeanAveragePrecision
preds = [
   dict(
...     boxes=torch.tensor([[258.0, 41.0, 606.0, 285.0]]),
...     scores=torch.tensor([0.536]),
...     labels=torch.tensor([0]),
...   )
... ]
>>> target = [
...   dict(
...     boxes=torch.tensor([[214.0, 41.0, 562.0, 285.0]]),
...     labels=torch.tensor([0]),
...   )
... ]
metric = MeanAveragePrecision()
metric.update(preds, target)


SyntaxError: ignored

In [ ]:
metric.compute()

In [ ]:
targets = list()

for image in test_dataset:
  targets.append(dict(boxes=image[1]['boxes'], labels=image[1]['labels']))


In [ ]:
target = [
   dict(
     boxes=torch.tensor([[214.0, 41.0, 562.0, 285.0]]),
     labels=torch.tensor([0]),
   )
]
target

In [ ]:
for image in os.listdir(test_images):
  img_path = test_images + '/' + image
  predictions = predict(model, img_path, 0.0)
  break
predictions

In [ ]:
import torchvision.transforms as transforms

def predict(model, img_path: str, threshold: float) -> tuple:
    transform = transforms.Compose([transforms.ToTensor()])
    img = Image.open(img_path)
    pred_img = transform(img)
    pred_img = pred_img.view(1, 3, pred_img.shape[1], pred_img.shape[2])

    preds = model(pred_img)
    outputs = [{k: v.to(torch.device('cpu')) for k, v in target.items()} for target in preds]

    boxes = outputs[0]['boxes'].data.cpu().numpy().astype(np.int32)
    scores = outputs[0]['scores'].data.cpu().numpy()
    labels = outputs[0]['labels'].data.cpu().numpy().astype(np.int32)

    boxes, scores, labels = get_relevant_scores(threshold, boxes, scores, labels)
    
    return boxes, scores, labels